In [1]:
from collections import defaultdict
import json
import os
from tqdm.auto import tqdm

from elasticsearch import Elasticsearch
from elasticsearch import helpers

hotpot_paragraphs = dict()
hotpot_title2id = dict()

index_name = 'enwiki-20171001-paragraph-3'
es = Elasticsearch(['10.60.0.59:9200'], timeout=30)
hotpot_query = {"query": {"term": {"for_hotpot": True}}}
hotpot_para_num = es.count(index=index_name, body=hotpot_query)['count']
for hit in tqdm(helpers.scan(es, query=hotpot_query, index=index_name), total=hotpot_para_num):
    para = hit['_source']
    para_id = para['para_id']
    title = para['title']
    if title in hotpot_title2id:
        print(f"Duplicate title: {title}")
    hotpot_title2id[title] = para_id
    hotpot_paragraphs[para_id] = {"title": title, "text": para['text'], "hyperlinks": para['hyperlinks']}
print(f"#Title in hotpot {len(hotpot_title2id):,d}")


#Title in hotpot 5,232,077


In [2]:
%%time

print(f"#Paragraph (non-empty) in hotpot {len(hotpot_paragraphs):,d}")
with open('data/hotpot-paragraph.tsv') as f:
    for line in f:
        segs = line.strip().split('\t')
        pid, text, title, sentence_spans = segs
        if pid != 'id':
            if pid not in hotpot_paragraphs:
                hotpot_paragraphs[pid] = {"title": title, "text": text, "sentence_spans": eval(sentence_spans), "hyperlinks": dict()}
            else:
                hotpot_paragraphs[pid]['sentence_spans'] = eval(sentence_spans)
print(f"#Paragraph in hotpot {len(hotpot_paragraphs):,d}")

#Paragraph (non-empty) in hotpot 5,232,077
#Paragraph in hotpot 5,344,953
CPU times: user 1min 13s, sys: 4.64 s, total: 1min 17s
Wall time: 1min 17s


In [25]:
from text_clean import clean_text


def norm_text(s):
    return ' '.join(clean_text(s).strip().split())


def find_final_sp(sp_paras, ans):
    if ans in f"{sp_paras[0]['title']} {sp_paras[0]['text']}" and ans not in f"{sp_paras[1]['title']} {sp_paras[1]['text']}":
        return sp_paras[0]['title']
    elif ans in f"{sp_paras[1]['title']} {sp_paras[1]['text']}" and ans not in f"{sp_paras[0]['title']} {sp_paras[0]['text']}":
        return sp_paras[1]['title']
    elif sp_paras[0]['title'] in sp_paras[1]['hyperlinks'] and sp_paras[1]['title'] not in sp_paras[1]['hyperlinks']:
        return sp_paras[0]['title']
    else:
        return sp_paras[1]['title']

data_dir = 'data/'
file_map = {
    "hotpot-train": "HotpotQA/hotpot_train_v1.1.json",
    "hotpot-dev": "HotpotQA/hotpot_dev_fullwiki_v1.json",
    "hotpot-test": "HotpotQA/hotpot_test_fullwiki_v1.json"
}

all_sp_titles = set()
for partition, raw_file in file_map.items():
    if 'test' in partition:
        continue
    with open(os.path.join(data_dir, raw_file)) as fin, open(os.path.join(data_dir, f"{partition}-with-neg.jsonl"), 'w') as fout:
        samples = json.load(fin)
        for sample in tqdm(samples):
            q_id = sample['_id']
            question = norm_text(sample['question'])
            answer = norm_text(sample['answer'])
            
            sp_titles = set()
            for sp in sample['supporting_facts']:
                sp_title = norm_text(sp[0])
                sp_titles.add(sp_title)
                all_sp_titles.add(sp_title)
            sp_titles = list(sp_titles)
            
            final_sp_title = None
            if sample['type'] != 'comparison':
                assert sample['type'] == 'bridge'
                final_sp_title = find_final_sp([hotpot_paragraphs[hotpot_title2id[t]] for t in sp_titles], answer)
                if final_sp_title == sp_titles[0]:
                    sp_titles = sp_titles[::-1]

            sp_ids = [hotpot_title2id[t] for t in sp_titles]            
            pos_paras = []
            query = {
                "query": {
                    "bool": {
                        "must": {"match": {"text": question}},
                        "filter": {"ids" : {"values": sp_ids}}
                    }
                }
            }
            sp_scores = dict()
            for hit in es.search(query, index_name)['hits']['hits']:
                sp_scores[hit['_id']] = hit['_score']
            for sp_id in sp_ids:
                pos_paras.append({
                    "para_id": sp_id,
                    "title": hotpot_paragraphs[sp_id]['title'],
                    "text": hotpot_paragraphs[sp_id]['text'],
                    "bm25_score": sp_scores[sp_id] if sp_id in sp_scores else 0.0
                })
            
            tfidf_neg_paras = []
            query = {
                "query": {
                    "bool": {
                        "must": {"match": {"text": question}},
                        "must_not" : {"ids" : {"values": sp_ids}},
                        "filter": {"term": {"for_hotpot": True}}
                    }
                },
                "size": 3
            }
            for hit in es.search(query, index_name)['hits']['hits']:
                para = hit['_source']
                tfidf_neg_paras.append({
                    "para_id": para['para_id'],
                    "title": para['title'],
                    "text": para['text'],
                    "bm25_score": hit['_score']
                })
            
            new_sample = {
                "_id": q_id,
                "question": question,
                "answer": answer,
                "sp_titles": sp_titles,
                "bridge": final_sp_title,
                "pos_paras": pos_paras,
                "neg_paras": [],
                "tfidf_neg": tfidf_neg_paras,
                "type": sample['type'],
                "level": sample['level']
            }
            fout.write(f"{json.dumps(new_sample, ensure_ascii=False)}\n")
            fout.flush()
print(f"#Title (supporting fact) in hotpot {len(all_sp_titles):,d}")


#Title (supporting fact) in hotpot 111,140
